### <b> Calculating the luminosity of a polytrope of index n </b>

In [4]:
from polytrope_model import lane_emden_sys, lane_emden_solver
import numpy as np

In [ ]:
# Thermodynamic functions at the center

def density(M, R, n):
    xi, y1, y2 = lane_emden_solver(n)
    a_n = -xi[-1]/(3*y2[-1])
    central_density = M/(4/3*np.pi*R**3) * a_n
    density = central_density * y1**n

    return density, central_density

def pressure(G, M, R, n):
    xi, y1, y2 = lane_emden_solver(n)
    c_n = 1 / (4 * np.pi * (n+1) * y2[-1]**2)
    central_pressure = (G*M**2)/R**4 * c_n
    pressure = central_pressure * y1**(n+1)

    return pressure, central_pressure

def temperature(G, M, R, X, Z, R_g, n): # R_g = gas constant, G  (define both as global variables?)
    xi, y1, y2 = lane_emden_solver(n)
    mu = 4/(5*X-Z+3)
    b_n = 1 / (n+1) * xi[-1] * (-y2[-1])
    central_temperature = mu*G*M/(R_g*R) * b_n
    temperature = central_temperature *  y1

    return temperature, central_temperature

In [ ]:
# General Thermodynamic functions

def density(M, R, n):
    c_rho = central_density(M,R,n)

def pressure():

def temperature():
